# Overview

Ref: [Calculating centred and non-centred volatility](http://vixandmore.blogspot.com/2009/12/calculating-centered-and-non-centered.html)

Stock Volatility measures how much a stock tends to move. There are many ways to calculate volatility, for e.g. through:

- Daily/Weekly/Monthly range
- Average True Range
- Standard Deviation

Standard Deviation is the most popular way to measure volatility. Standard Deviation for stock price can be computed in multiple ways. Let us go through them.

In [1]:
import pandas as pd
import math

# computing SD of a series of values
data = [161.47, 159.86, 159.27, 159.98, 159.78, 157.21, 157.5, 157.86, 160.95, 161.6, 159.85, 
         157.48, 155.32, 160.43, 159.45, 158.19, 155.78, 154.96, 156.53, 149.46, 148.15] # gives sd of 1.59%

s = pd.Series(data, name='data')
df = pd.DataFrame(s)

## 1) From math library's standard deviation computation.

The code for this is as follows:


In [2]:
asd = df.data.std()
amean = df.data.mean()
dict(asd=asd, amean=amean)

{'asd': 3.533283849608569, 'amean': 157.67047619047622}

Arithmetic standard deviation makes sense for a stable mean. However, in stock prices the price keeps current price keeps changing. So it is better to use **geometric standard deviation** or GSD. 

Here are different ways of getting GSD:

## 2a) From daily log returns assuming zero mean and averaging squares
Ref: [this Quora link](https://www.quora.com/How-do-you-calculate-the-standard-deviation-for-a-stock)

Formula for geometric standard deviation is represented by:
     
gsd1 = $\exp{\Big(\sqrt{{\frac{\sum_{1}^n (\ln\frac{x[n]}{x[n-1]})^{2}}{n}}}}\Big)$

Geometric standard deviation is computed over geometric mean, which is represented by the formula:

gm = $\left (\prod_{a=1}^{b}x_i  \right )^{\frac{1}{n}} = \sqrt[n]{x_1x_2...x_n}$

The upper and lower values of 1 GSD are computed by dividing and multiplying geometric mean with the geometric standard deviation

The code for these are as follows:

```
import math
from scipy.stats.mstats import gmean
sdmult = 2 # no of standard deviations
gsd1 = math.exp(math.sqrt(df.data.rolling(2).apply(lambda x: math.log(x[0]/x[-1])**2, raw=True).mean()))
gm = gmean(df)
glo1 = gm*(1-(gm-gm/gsd1)/gm*sdmult) # gm/gsd1 ... but taking sdmult into consideration
ghi1 = gm*(1+(gm*gsd1-gm)/gm*sdmult) # gm*gsd1 ... but taking sdmult into consideration

```



In [7]:
import math
from scipy.stats.mstats import gmean
sdmult = 2 # no of standard deviations
gsd1 = math.exp(math.sqrt(df.data.rolling(2).apply(lambda x: math.log(x[0]/x[-1])**2, raw=True).mean()))
gm = gmean(df)
glo1 = gm*(1-(gm-gm/gsd1)/gm*sdmult) # gm/gsd1 ... but taking sdmult into consideration
ghi1 = gm*(1+(gm*gsd1-gm)/gm*sdmult) # gm*gsd1 ... but taking sdmult into consideration

dict(gm=gm, gsd1=gsd1, glo1=glo1, ghi1=ghi1)

{'gm': array([157.63194626]),
 'gsd1': 1.0160309183814213,
 'glo1': array([152.65771798]),
 'ghi1': array([162.68591599])}

## 2b) From daily log returns without squaring and square-rooting

The formula for this is:

sd2 = ${\exp(\sigma\Big({{\ln(\frac{x[n]}{x[n-1]})}}\Big))}$

This equation gives a slightly lesser band than taking rooting...

In [8]:
import math
from scipy.stats.mstats import gmean
sdmult = 2 # no of standard deviations
gsd2 = math.exp(df.data.rolling(2).apply(lambda x: math.log(x[0]/x[-1]), raw=True).std())
glo2 = gm*(1-(gm-gm/gsd2)/gm*sdmult) # gm/gsd2 ... but taking sdmult into consideration
ghi2 = gm*(1+(gm*gsd2-gm)/gm*sdmult) # gm*gsd2 ... but taking sdmult into consideration

dict(gm=gm, gsd2=gsd2, glo2=glo2, ghi2=ghi2)

{'gm': array([157.63194626]),
 'gsd2': 1.015831867370207,
 'glo2': array([152.71851886]),
 'ghi2': array([162.62316239])}

## 2c) Centred Historical Volatiity using look-back

[This is the most preferred approach](http://vixandmore.blogspot.com/2009/12/calculating-centered-and-non-centered.html) of using SD for volatility. The steps are as follows:

1. Select a desired lookback period in trading days (lookback period)
2. Gather closing prices for the full lookback period, plus one additional day (lookback +1)
4. Calculate the daily close-to-close price changes in a security for each day in the lookback period (daily change)
5. Determine the natural log of each daily percentage change (log of daily changes)
6. Calculate the mean of all the natural logs of the closing prices for the lookback period (log lookback mean)
7. For each day, subtract the lookback mean from the log of daily changes (daily difference)
8. Square all the differences between the mean and the daily change (daily variance)
9. Sum all the squares of the differences (sum of variances)
10. Divide the sum of the squares of the variances by the lookback period (lookback variance)
11. Take the square root of the lookback variance (historical volatility, expressed as a standard deviation)
12. Compute the high and low bands for the lookback period
